In [2]:
import seaborn as sns
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from mpl_toolkits import mplot3d
import cv2
import time

histograms in seaborn here: https://www.machinelearningplus.com/plots/matplotlib-histogram-python-examples/

In [2]:
save_location =  "../FLIR_Camera"
pdfCombined_figs_location  = "../FLIR_Camera/fastFlow_ShallowH_test1_centeredCame_flume_LookAngle_35Deg_PDFCOMBINED_FIGURES"

In [3]:
BEDFORMS = ['corals', 'canopy', 'rocks', 'dunes']
colors = {'corals':'dodgerblue','canopy':'green', 'rocks':'orange', 'dunes':'brown'}
bed_data = {}
for BEDFORM in BEDFORMS:
    data_filename = BEDFORM + "_fastFlow_ShallowH_test1.pickle"
    with open(os.path.join(save_location,data_filename), 'rb') as handle:
        all_data_dict_loaded = pickle.load(handle)
    bed_data[BEDFORM] = all_data_dict_loaded

In [4]:
NUM_FRAMES = min([bed_data[BEDFORM]['all_height_maps'].shape[2] for BEDFORM in BEDFORMS])
H_MIN = 0.7*min([np.min(bed_data[BEDFORM]['all_height_maps']) for BEDFORM in BEDFORMS])
H_MAX = 0.7*max([np.max(bed_data[BEDFORM]['all_height_maps']) for BEDFORM in BEDFORMS])
SX_MIN = -1#min([np.min(bed_data[BEDFORM]['all_sx_maps']) for BEDFORM in beds])
SX_MAX = 1#max([np.max(bed_data[BEDFORM]['all_sx_maps']) for BEDFORM in beds])
SY_MIN = -1#min([np.min(bed_data[BEDFORM]['all_sy_maps']) for BEDFORM in beds])
SY_MAX = 1#max([np.max(bed_data[BEDFORM]['all_sy_maps']) for BEDFORM in beds])

## Plot pdf of the height field and slopes

In [5]:
sns.set_style("white")
kwargs = dict(hist_kws={'alpha':.5}, kde_kws={'linewidth':2})
height_df = {}
sx_df = {}
sy_df = {}
for i in tqdm(range(NUM_FRAMES)):#NUM_FRAMES
    
    fig = plt.figure(figsize =(30, 12))
    
    for BEDFORM in BEDFORMS:
        height_map = bed_data[BEDFORM]['all_height_maps'][:,:,i]
        height_df[BEDFORM] = pd.DataFrame(np.real(height_map).ravel())
        
        sx_map = bed_data[BEDFORM]['all_sx_maps'][:,:,i]
        sx_df[BEDFORM] = pd.DataFrame(sx_map.ravel())
        
        sy_map = bed_data[BEDFORM]['all_sy_maps'][:,:,i]
        sy_df[BEDFORM] = pd.DataFrame(sy_map.ravel())
    

    ax0 = fig.add_subplot(1, 3, 1)
    ax0.set_title('Normalized histogram + KDF ($\eta$)', fontsize = 34)
    for BEDFORM in BEDFORMS:
        sns.distplot(height_df[BEDFORM], ax=ax0, color=colors[BEDFORM], label = BEDFORM, **kwargs)
    ax0.set_xlabel("$\eta(x,y)$", fontsize = 24)
    ax0.legend(fontsize = 24)
    ax0.set_xlim([H_MIN, H_MAX])
    
    ax1 = fig.add_subplot(1, 3, 2)
    ax1.set_title('Normalized histogram + KDF ($s_x$)', fontsize = 34)
    for BEDFORM in BEDFORMS:
        sns.distplot(sx_df[BEDFORM], ax=ax1, color=colors[BEDFORM], label = BEDFORM, **kwargs)
    ax1.set_xlabel("$s_x(x,y)$", fontsize = 24)
    ax1.legend(fontsize = 24)
    ax1.set_xlim([SX_MIN, SX_MAX])
    
    ax2 = fig.add_subplot(1, 3, 3)
    ax2.set_title('Normalized histogram + KDF ($s_y$)', fontsize = 34)
    for BEDFORM in BEDFORMS:
        sns.distplot(sy_df[BEDFORM], ax=ax2, color=colors[BEDFORM], label = BEDFORM, **kwargs)
    ax2.set_xlabel("$s_y(x,y)$", fontsize = 24)
    ax2.legend(fontsize = 24)
    ax2.set_xlim([SY_MIN, SY_MAX])
    
    plt.savefig(os.path.join(pdfCombined_figs_location, 'fig_' + str(i).zfill(6) + '.png'), dpi = 80)
    plt.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [09:01<00:00,  7.80s/it]


In [6]:
os.path.exists(pdfCombined_figs_location)

True